<a href="https://colab.research.google.com/github/vishnupriyamuddamsetty/pancreaticcancer-mlproject/blob/main/pancreaticcancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelBinarizer
import warnings
!pip install tensorflow_addons
sns.set_theme(style="darkgrid")
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 10.7 MB/s eta 0:00:00


In [ ]:
og_data = pd.read_csv('Debernardi et al 2020 data.csv')
og_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   sample_id                590 non-null    object 
 1   patient_cohort           590 non-null    object 
 2   sample_origin            590 non-null    object 
 3   age                      590 non-null    int64  
 4   sex                      590 non-null    object 
 5   diagnosis                590 non-null    int64  
 6   stage                    199 non-null    object 
 7   benign_sample_diagnosis  208 non-null    object 
 8   plasma_CA19_9            350 non-null    float64
 9   creatinine               590 non-null    float64
 10  LYVE1                    590 non-null    float64
 11  REG1B                    590 non-null    float64
 12  TFF1                     590 non-null    float64
 13  REG1A                    306 non-null    float64
dtypes: float64(6), int64(2), o

In [ ]:
og_data.head()

,sample_id,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,S1,Cohort1,BPTB,33,F,1,NaN,NaN,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,S10,Cohort1,BPTB,81,F,1,NaN,NaN,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,S100,Cohort2,BPTB,51,M,1,NaN,NaN,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,S101,Cohort2,BPTB,61,M,1,NaN,NaN,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,S102,Cohort2,BPTB,62,M,1,NaN,NaN,9.0,0.21489,0.000860,65.54000,41.088000,NaN


Data preprocessing

In [ ]:

data = og_data.drop(columns=['sample_id', 'patient_cohort', 'sample_origin',
                             'stage', 'benign_sample_diagnosis'])
data.head(10)

,age,sex,diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,33,F,1,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,81,F,1,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,51,M,1,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,61,M,1,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,62,M,1,9.0,0.21489,0.000860,65.54000,41.088000,NaN
5,53,M,1,NaN,0.84825,0.003393,62.12600,59.793000,NaN
6,70,M,1,NaN,0.62205,0.174381,152.27700,117.516000,NaN
7,58,F,1,11.0,0.89349,0.003574,3.73000,40.294000,NaN
8,59,F,1,NaN,0.48633,0.001945,7.02100,26.782000,NaN
9,56,F,1,24.0,0.61074,0.278778,83.92800,19.185000,NaN


In [ ]:
data.replace({'sex': {'M': 1, 'F': 0}}, inplace=True)
data.head(10)


,age,sex,diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,33,0,1,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,81,0,1,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,51,1,1,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,61,1,1,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,62,1,1,9.0,0.21489,0.000860,65.54000,41.088000,NaN
5,53,1,1,NaN,0.84825,0.003393,62.12600,59.793000,NaN
6,70,1,1,NaN,0.62205,0.174381,152.27700,117.516000,NaN
7,58,0,1,11.0,0.89349,0.003574,3.73000,40.294000,NaN
8,59,0,1,NaN,0.48633,0.001945,7.02100,26.782000,NaN
9,56,0,1,24.0,0.61074,0.278778,83.92800,19.185000,NaN


In [ ]:
data['plasma_CA19_9'] = data['plasma_CA19_9'].interpolate(method='polynomial', order=2)
data['REG1A'] = data['REG1A'].interpolate(method='polynomial', order=2)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590 entries, 0 to 589
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            590 non-null    int64  
 1   sex            590 non-null    int64  
 2   diagnosis      590 non-null    int64  
 3   plasma_CA19_9  590 non-null    float64
 4   creatinine     590 non-null    float64
 5   LYVE1          590 non-null    float64
 6   REG1B          590 non-null    float64
 7   TFF1           590 non-null    float64
 8   REG1A          590 non-null    float64
dtypes: float64(6), int64(3)
memory usage: 41.6 KB


In [ ]:
X = data.loc[:, data.columns != 'diagnosis'].to_numpy()
y = data['diagnosis'].to_numpy()
X.shape, y.shape

((590, 8), (590,))

In [ ]:
lb = LabelBinarizer()
y_bin = lb.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size = 0.2, random_state = 0)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((472, 8), (118, 8), (472, 3), (118, 3))

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_std

array([[-6.60594114e-01, -9.42337519e-01, -1.86294560e-01, ...,
        -4.63872971e-01, -4.07503170e-01, -1.41147536e-01],
       [-1.36056034e-01,  1.06119090e+00, -1.04270474e-01, ...,
        -5.02539472e-01,  9.90995251e-02, -1.75694340e-01],
       [-4.35792080e-01, -9.42337519e-01,  2.82361446e-02, ...,
        -4.33380556e-01, -1.07776307e-01, -1.21985787e-01],
       ...,
       [-1.40993423e+00,  1.06119090e+00, -1.84694157e-01, ...,
         7.49886973e-02, -2.54014377e-01,  1.80955852e+00],
       [ 1.13782216e+00, -9.42337519e-01, -1.87127227e-01, ...,
        -4.52335520e-01, -4.46042655e-01,  3.92698967e-01],
       [-4.35792080e-01,  1.06119090e+00,  6.06665472e+00, ...,
        -1.23983868e-02, -1.64866375e-01, -5.37506706e-03]])

In [ ]:
from tensorflow.keras.layers import Conv1D,MaxPool1D,Dense,Dropout,BatchNormalization,Flatten
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad,SGD
from keras.optimizers import SGD

import tensorflow_addons as tfa

from tensorflow.keras import layers

In [ ]:
metrics = ['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),tf.keras.metrics.RootMeanSquaredError(),
           tf.keras.metrics.MeanAbsolutePercentageError(),tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.AUC(curve='ROC'),tfa.metrics.F1Score(num_classes=3),
           tf.keras.metrics.AUC(curve='PR')]

In [ ]:
# Part 2 - Building the ANN
# Initializing the ANN
ann = tf.keras.models.Sequential()
# Adding the input layer and the first hidden layer
ann.add(tf.keras.layers.Dense(units=8, activation='relu'))

# Adding the second hidden layer
# ann.add(tf.keras.layers.Dense(units=10, activation='sigmoid'))
#ann.add(Dropout(0.5))

ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(Dropout(0.5))

# Adding the output layer
ann.add(tf.keras.layers.Dense(units=3, activation='softmax'))

# Part 3 - Training the ANN

opt = Adam(lr=0.001)
# Compiling the ANN
ann.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics =metrics)

# Training the ANN on the Training set
history_ann = ann.fit(X_std, y_train,batch_size=8, epochs=200,validation_data=(X_test_std, y_test))


# Part 4 - Making the predictions and evaluating the model

# Predicting the result of a single observation

Epoch 1/200
59/59 [==============================] - 2s 12ms/step - loss: 1.1707 - accuracy: 0.2966 - precision: 0.4509 - recall: 0.1322 - root_mean_squared_error: 0.4717 - mean_absolute_percentage_error: 210517728.0000 - mean_absolute_error: 0.4210 - auc: 0.5658 - f1_score: 0.3475 - auc_1: 0.4150 - val_loss: 1.1433 - val_accuracy: 0.2797 - val_precision: 0.1667 - val_recall: 0.0085 - val_root_mean_squared_error: 0.4802 - val_mean_absolute_percentage_error: 223982672.0000 - val_mean_absolute_error: 0.4480 - val_auc: 0.4807 - val_f1_score: 0.2820 - val_auc_1: 0.3132
Epoch 2/200
59/59 [==============================] - 0s 4ms/step - loss: 1.1146 - accuracy: 0.3326 - precision: 0.3457 - recall: 0.0593 - root_mean_squared_error: 0.4752 - mean_absolute_percentage_error: 218211696.0000 - mean_absolute_error: 0.4364 - auc: 0.5414 - f1_score: 0.3317 - auc_1: 0.3635 - val_loss: 1.0852 - val_accuracy: 0.3898 - val_precision: 0.7143 - val_recall: 0.0424 - val_root_mean_squared_error: 0.4686 - val